# Location vs Distance

In [ ]:
import json
import re


import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect


from checklist.pred_wrapper import PredictorWrapper

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")


%run utils.ipynb

# Initialize random seed
import random
#import torch
seed = 423
random.seed(seed)
#torch.manual_seed(seed)

## Creating  tests

In [ ]:
def generate_loc_vs_dir(editor, nr_of_samples):
    
    capability = 'capability3_loc_vs_dir'
    test_type  = 'MFT'
    function_expected = one_arg_expected
    nr_of_samples = nr_of_samples
    tests = []


    # create test material dictionary
    test_material = {}
    test_material['dir'] = {}
    test_material['loc'] = {}

    test_material['dir']['capability']  = capability
    test_material['dir']['test_type']   = test_type
    test_material['dir']['expected_fn'] = function_expected

    test_material['loc']['capability']  = capability
    test_material['loc']['test_type']   = test_type
    test_material['loc']['expected_fn'] = function_expected

    test_material['dir']['tests'] = {}
    test_material['loc']['tests'] = {}

    tmd = test_material['dir']['tests']
    tml = test_material['loc']['tests']



    ## Sentences Dir


    # examples taken from https://verbs.colorado.edu//~mpalmer/projects/ace/PBguidelines.pdf

    sentences_list = ['\"What sector is stepping forward to pick up the slack ?\" he asked.', 
                     'Workers dumped large burlap sacks of the imported material into a huge bin.',
                     '\"That response annoyed Rep. Markey\", House aides said, and the congressman snapped back that there had been enough studies of the issue and that it was time for action on the matter .']



    # test 1
    # '\"What sector is stepping forward to pick up the slack ?\" he asked.'

    t = 'test1_directional'
    tmd[t] = {}
    tmd[t]['sentence']   = '\"{question_word_s_one} sector {aux_tenses} {predicate_stepping} {dir_s_one} to pick up the slack ?\" {pronoun} asked.'
    tmd[t]['target']     = 'dir_s_one'
    tmd[t]['arg_target'] = 'ARGM-DIR'
    tmd[t]['predicate']  = 'predicate_stepping'


    editor.add_lexicon(name='pronoun', 
                       values=['I', 'you', 'he', 'she', 'it', 'we', 'they'], 
                       overwrite=True)

    editor.add_lexicon(name='dir_s_one', 
                       values=['forward', 'aside'], 
                       overwrite=True)

    editor.add_lexicon(name='aux_tenses', 
                       values=['is', 'was', 'had been', 'will be', 'will have been'], 
                       overwrite=True )

    editor.add_lexicon(name='question_word_s_one', 
                       values=['What', 'Which', 'Which kind of'], 
                       overwrite=True )

    editor.add_lexicon(name='predicate_stepping', 
                       values=['stepping'], 
                       overwrite=True )


    ## Sentences Loc

    # test 2
    # 'Areas of the factory were particularly dusty where the crocidolite was used.'

    t = 'test1_location'
    tml[t] = {}
    tml[t]['sentence']   = 'Areas of the {place} were particularly dusty {loc_s_one} the {thing_to_use} was {predicate_used}.'
    tml[t]['target']     = 'loc_s_one'
    tml[t]['arg_target'] = 'ARGM-LOC'
    tml[t]['predicate']  = 'predicate_used'

    editor.add_lexicon(name='loc_s_one', 
                       values=['where'], 
                       overwrite=True)

    editor.add_lexicon(name='thing_to_use', 
                       values=['crocidolite', 'aluminiom', 'corn', 'machine', 'drill', 'saw', 'wood', 'mineral', 'asbestos', 'Acuminite', 'Jadeite'], 
                       overwrite=True)

    editor.add_lexicon(name='place', 
                       values=['factory', 'room', 'hall', 'workshop', 'mill', 'firm', 'shop', 'laboratory', 'house', 'mint', 'werehouse'], 
                       overwrite=True)

    editor.add_lexicon(name='predicate_used', 
                       values=['used'], 
                       overwrite=True )

    
    
    for key in test_material.keys():

        new_tests = create_tests(test_material[key], nr_of_samples)
        
        for test in new_tests:
            tests.append(test)

    return tests